# ECCC Hydat streamflow dawta
Andrew Ireson, 22 March 2020

Instructions... just run this first... then you should be able to figure it out.

Do not expect these data to be too up to date!

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import sqlite3
from saveload import saveload as sl

In [ ]:
def GetData(prov,searchterm):
    db='Hydat_flow/Hydat.sqlite3'
    con=sqlite3.connect(db)
    
    tables=pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';",con)
    stations=pd.read_sql_query("SELECT * from STATIONS", con)
    
    i=[ind for ind,x in enumerate(stations['PROV_TERR_STATE_LOC']) if x==prov]
    stn=stations.iloc[i]
    
    i=[ind for ind,x in enumerate(stn['STATION_NAME']) if searchterm.lower() in x.lower()]
    stn_name=stn.iloc[i]['STATION_NAME'].values
    if len(stn_name)>1:
        print('Your search returned more than one station... \nSee list below and refine your search:\n')
        print(stn_name)
        return [],[]
    
    elif len(stn_name)==0:
        print('Your search returned nothing... try again')
        return [],[]
    
    else:
        print('Returned data for ' + stn_name[0])
        stn_no=stn['STATION_NUMBER'].iloc[i].values[0]

        data=pd.read_sql_query('SELECT * from DLY_FLOWS where STATION_NUMBER="%s"'%stn_no, con)

        qd=GetDaily(data)
        qm=GetMonthly(data)
    
        return qd,qm

def GetDaily(data):
    
    years=data['YEAR'].values
    months=data['MONTH'].values
    days=data['NO_DAYS'].values
    
    # Daily flow
    flow=np.zeros((len(days),31))
    for i in range(31):
        flow[:,i]=data['FLOW%d'%(i+1)].values

    # reshape flows onto a single column
    Q=np.array([])
    for i,d in enumerate(days):
        Q=np.hstack((Q,flow[i,:d]))

    # get datetimes
    tday=[]
    for i in range(len(days)):
        y=years[i]
        m=months[i]
        for day in range(1,days[i]+1):
            tday.append(pd.Timestamp(year=y,month=m,day=day))

    qd=pd.DataFrame(index=tday)
    qd['Q']=Q
    
    # This line replaces any gaps with nans
    qd=qd.resample('D').first()
    
    return qd

def GetMonthly(data):
    
    years=data['YEAR'].values
    months=data['MONTH'].values
    days=data['NO_DAYS'].values
    
    # Mean monthly flow
    meanflow=data['MONTHLY_MEAN'].values
    tmon=[pd.Timestamp(year=y,month=m,day=1) for y,m in zip(years,months)]
    qm=pd.DataFrame(index=tmon)
    qm['Q']=meanflow
    qm=qm.resample('MS').first()
    
    return qm

In [ ]:
# ONLY RUN THIS ONCE - Downloads the Hydat database - large file
import os.path

if os.path.isfile('Hydat_flow/Hydat.sqlite3'):
    print('You already have the Hydat SQL database... nothing downloaded')
else:
    import urllib.request
    import zipfile

    # Download (this will take awhile, and will save the file Hydat.zip)
    # Note - you may need to check to see if the file name has been updated - go here https://collaboration.cmc.ec.gc.ca/cmc/hydrometrics/www
    url='https://collaboration.cmc.ec.gc.ca/cmc/hydrometrics/www/Hydat_sqlite3_20200715.zip'
    urllib.request.urlretrieve(url, './Hydat_flow/Hydat.zip')

    # Unzip hydat.zip which will extract the file Hydat.sqlite3
    with zipfile.ZipFile('./Hydat_flow/Hydat.zip', 'r') as zip_ref:
        zip_ref.extractall('Hydat_flow')

In [ ]:
qd,qm=GetData('SK','at saskatoon')

In [ ]:
# Plot flows
ax=qd.resample('MS').mean().plot(marker='o')
qm.plot(ax=ax,marker='.')
pl.legend(['Daily resampled','Monthly'])

In [ ]:
# Plot just one year:
qd['2005'].plot()

In [ ]:
# Plot data for a number of years, against day of year
for y in range(2000,2019):
    pl.plot(qd[str(y)].index.dayofyear,qd[str(y)])

In [ ]:
# Save the data
sl.save(qd,'SouthSask_Saskatoon')